<a href="https://colab.research.google.com/github/spatank/CIS-530/blob/master/Homework%206/hw6_skeleton_RIF_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and unzip files

In [1]:
#Download and unzip files
!pip3 install scikit-learn
!wget http://computational-linguistics-class.org/homework/nn-lms/cities_test.txt
!wget http://computational-linguistics-class.org/homework/nn-lms/cities_val.zip
!wget http://computational-linguistics-class.org/homework/nn-lms/cities_train.zip
!sudo apt-get install unzip
!unzip cities_val.zip 
!unzip cities_train.zip 
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torch torchvision
  
import torch
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

--2020-02-23 16:51:50--  http://computational-linguistics-class.org/homework/nn-lms/cities_test.txt
Resolving computational-linguistics-class.org (computational-linguistics-class.org)... 185.199.110.153
Connecting to computational-linguistics-class.org (computational-linguistics-class.org)|185.199.110.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10973 (11K) [text/plain]
Saving to: ‘cities_test.txt’

cities_test.txt     100%[===================>]  10.72K  --.-KB/s    in 0s      

2020-02-23 16:51:50 (227 MB/s) - ‘cities_test.txt’ saved [10973/10973]

--2020-02-23 16:51:51--  http://computational-linguistics-class.org/homework/nn-lms/cities_val.zip
Resolving computational-linguistics-class.org (computational-linguistics-class.org)... 185.199.110.153
Connecting to computational-linguistics-class.org (computational-linguistics-class.org)|185.199.110.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7558 (7.4K) [application/zip]
Sav

In [2]:
print(print(torch.__version__))

1.0.1
None


# Verify file download

In [3]:
#Verfiy file download
!head train/af.txt
!printf "\n"
!head val/af.txt
!printf "\n"
!head cities_test.txt
!printf "\n"
#Verify CUDA acceleration should print cuda:0
print(device)

garavshakh
kalishli
sarban kelay
dahan-e surunah
jahannum
morafeno
iskatel
zeri-chinar
taftebole
dzhafarabat

sikhtopa
laferronay
cham ghafur
souk tafetecht
panqash
siah sar bala
barah khan kili
vetob
kacari
kolga'i-i-bala

imam bakhsh amar
firoz laghari
glufishevo
qal`eh-ye ghazanfarkhani
nikolinalevada
kolkko
mbrostar i fierit
kharabanan-e sofla
stein bei nurnberg
jahangir bara

cuda:0


# Additional Imports

In [0]:
import codecs

# Paths

In [0]:
baseDir = '/content/gdrive/Shared drives/cis530_hw6/'
vectors_baseDir = '/content/gdrive/Shared drives/vectors/'

# Mount google drive

In [6]:
# Mount your google drive. 
# Use this to save your PyTorch model for submission
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/Shared drives/cis530_hw6/'
!ls
#!mkdir /content/gdrive/My\ Drive/cis530_hw6
#Test drive access. 
#You should have a test.txt under a new folder cis530_hw6 in your Google drive
with open('/content/gdrive/Shared drives/cis530_hw6/test.txt', 'w') as f:
  f.write('This is a test!')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/Shared drives/cis530_hw6
cities_test.txt     hw6_skeleton_RIF_edit.ipynb  test.txt	val
data		    hw6_SP_edit.ipynb		 text_file.txt
hw6_skeleton.ipynb  skeleton			 train


# City Names Task (adapted from Baby Names Tutorial)

## Preparing The Data

### Dictionaries

In [7]:
abvr_to_country_dict = {'af':'Afghanistan', 'cn':'China', 'de': 'Germany', 'fi':'Finland', 'fr':'France', 'in':'India', 'ir':'Iran', 'pk':'Pakistan', 'za':'South Africa'}
country_to_abv_dict = dict([(y,x) for x,y in abvr_to_country_dict.items()])
print(country_to_abv_dict)

abvr_to_filenames_dict = {'af':'af.txt', 'cn':'cn.txt', 'de':'de.txt', 'fi':'fi.txt', 'fr':'fr.txt', 'in':'in.txt', 'ir':'ir.txt', 'pk':'pk.txt', 'za':'za.txt'}


{'Afghanistan': 'af', 'China': 'cn', 'Germany': 'de', 'Finland': 'fi', 'France': 'fr', 'India': 'in', 'Iran': 'ir', 'Pakistan': 'pk', 'South Africa': 'za'}


### Stub Code

In [8]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print("this is findFiles: ", findFiles(baseDir + 'train/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print("test: ", unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = codecs.open(filename, "r",encoding='utf-8', errors='ignore').read().strip().split('\n')
    # lines = open(filename, encoding='utf-8', errors='ignore').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles(baseDir + 'train/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)
print("this is all_categories: ", all_categories)

this is findFiles:  ['/content/gdrive/Shared drives/cis530_hw6/train/fi.txt', '/content/gdrive/Shared drives/cis530_hw6/train/de.txt', '/content/gdrive/Shared drives/cis530_hw6/train/cn.txt', '/content/gdrive/Shared drives/cis530_hw6/train/af.txt', '/content/gdrive/Shared drives/cis530_hw6/train/ir.txt', '/content/gdrive/Shared drives/cis530_hw6/train/pk.txt', '/content/gdrive/Shared drives/cis530_hw6/train/fr.txt', '/content/gdrive/Shared drives/cis530_hw6/train/in.txt', '/content/gdrive/Shared drives/cis530_hw6/train/za.txt']
test:  Slusarski
this is all_categories:  ['fi', 'de', 'cn', 'af', 'ir', 'pk', 'fr', 'in', 'za']


### Category Lines test

In [9]:
print(category_lines[country_to_abv_dict['India']][:5])
print(category_lines[country_to_abv_dict['China']][:5])
print("this is category_lines: ", category_lines)

['tagegin', 'kheda', 'trichy', 'palaina', "sainthilairede l'aude"]
['yungming', 'xingzhuang', 'liren', 'hongjiaotian', 'guanrenling']
this is category_lines:  {'fi': ['navala', 'birrlufig', 'kellarpelto', 'myyra', 'tammi', 'butterfield', 'mujula', 'finejdet', 'gav sefid koochak', 'piilola', 'jylanki', 'rafin guzuma', 'kanapera', 'goth mitha pitafi', 'chalfir', 'kahrize lotfi', 'sitio joao figura', 'jollofin', 'fiefall', 'qaleh sufi', 'urfique', 'otere ofie', 'vandaparken', 'kokonmaki', 'vafikhorion', 'rufiri', 'tiffi', 'puente fierro', 'fickle hill', 'kassila', 'kalasarfirazkhan', 'lievisenmaki', 'herrfors', 'vilobacka', 'finca caoba', 'kumpula', 'mahe firuzan', 'karjalaisenniemi', 'loukinainen', 'nunnanoja', 'unterstampfing', 'ikot offiong ambai', 'yttersundom', 'brusas', 'chak six hundred seventyfourfifteen', 'pihkakoski', 'pylkonmaki', 'sunfield mobile home park', 'qalahye sefid', 'tunyifilidi', 'ilmolahti', 'pukala', 'marfinka', 'pehto', 'hakovaara', 'konkefing', 'kokinvaara', 'fie

### Turning Names into Tensors

In [10]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('j'))

print(lineToTensor('jahannum').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([8, 1, 57])


## Creating the network

### The RNN

In [0]:
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

### Letter to Tensor

In [0]:
input = letterToTensor('A')
hidden =torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

### Line to Tensor

In [13]:
input = lineToTensor('Albert')
# print("input: ", input)
# print("input[0]: ", input[0])
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[-2.1832, -2.2446, -2.2866, -2.1435, -2.2659, -2.2315, -2.1763, -2.0794,
         -2.1810]], grad_fn=<LogSoftmaxBackward>)


## Preparing For Training

In [14]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('in', 7)


### get training example

In [18]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    print("here: ", [all_categories.index(category)])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

here:  [7]
category = in / line = ban thing tum
here:  [4]
category = ir / line = khidhira
here:  [8]
category = za / line = bachakan ahmadzai
here:  [0]
category = fi / line = fidushkuh
here:  [5]
category = pk / line = toti machhi
here:  [2]
category = cn / line = chejiatan
here:  [8]
category = za / line = sitio joao felizardo
here:  [5]
category = pk / line = mota din
here:  [3]
category = af / line = khushkukharab
here:  [6]
category = fr / line = undurein


## Training The Network

Each loop of training will:

1. Create input and target tensors
2. Create a zeroed initial hidden state
3. Read each letter in and
4. Keep hidden state for next letter
5. Compare final output to target
6. Back-propagate
7. Return the output and loss

### Train

In [0]:
learning_rate = 0.001 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

## Gather Data and Plot

In [17]:
import math
import time

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

NameError: ignored

### Plot Results

In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

### Evaluating the Results

In [0]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

## Run predict()

In [0]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict('khabibkoruna') ## af
predict('gongfu') ## cn
predict('saint-julien-de-l\'herms') ## de
predict('vilobacka') ## fi
predict('premery') ## fr
predict('jirsar-e vishkah') ## ir
predict('nagad') ## in
predict('zamet') ## za
predict('typkysyr') ## pk

# Write output files (all methods)


In [0]:
from datetime import datetime
import csv

In [0]:
## ADDED
def write_file(filename, results):
  '''
    Input: List ['this is an', 'example']
    Output: This overwirtes file every time, does not append
  '''
  dateTimeObj = datetime.now()
  results.insert(0,'******* New File ' + str(dateTimeObj) + '***********')
  with open(filename,'w') as f:
      for item in results:
          f.write(str(item) + '\n')

  f.close()

In [0]:
## ADDED
def append_to_output_file(filename, results):
  '''
    Input: List ['this is an', 'example']
    Output: Keeps appending to same file (does NOT overwrite)
  '''
  dateTimeObj = datetime.now()
  results.insert(0,'******* New File ' + str(dateTimeObj) + '***********')
  
  ## write to output file
  with open(filename,'a') as f:
      results.insert(0, "*******  Beginning of new File  ********")
      print(type(results))
      for item in results:
          f.write(str(item) + '\n')

  f.close()

In [0]:
## ADDED
def write_outfile(filename, results):
  '''
    Input: String
    Output: Keeps appending to same file (does NOT overwrite)
  '''
  dateTimeObj = datetime.now()

  with open(filename,'a') as f:
      f.write('******* Beginning of file ' + str(dateTimeObj) +  '********\n')
      f.write(results + '\n')

  f.close()

In [0]:
## ADDED
def write_outfile_with_modelname(filename, results, modelname):
  '''
    Input: String, modelname to be printed with results
    Output: Keeps appending to same file (does NOT overwrite)
  '''
  dateTimeObj = datetime.now()

  with open(filename,'a') as f:
      f.write(modelname + '\n')
      f.write('******* Beginning of file ' + str(dateTimeObj) +  '********\n')
      f.write(results + '\n')

  f.close()

# main_classify.py

In [0]:
#main_classify.py
import codecs
import copy
import math
import random
import string
import time
import numpy as np
import torch
from sklearn.metrics import accuracy_score
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

'''
Don't change these constants for the classification task.
You may use different copies for the sentence generation model.
'''
## global variables
languages = ["af", "cn", "de", "fi", "fr", "in", "ir", "pk", "za"] ## this is all_categories:  ['fi', 'de', 'cn', 'af', 'ir', 'pk', 'fr', 'in', 'za']
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)
n_categories_train = 0
all_categories_train = []
n_categories_val = 0
all_categories_val = []

## root directory
baseDir = '/content/gdrive/Shared drives/cis530_hw6/'

## helpful dictionaries
abvr_to_country_dict = {'af':'Afghanistan', 'cn':'China', 'de': 'Germany', 'fi':'Finland', 'fr':'France', 'in':'India', 'ir':'Iran', 'pk':'Pakistan', 'za':'South Africa'}
country_to_abv_dict = dict([(y,x) for x,y in abvr_to_country_dict.items()])
abvr_to_filenames_dict = {'af':'af.txt', 'cn':'cn.txt', 'de':'de.txt', 'fi':'fi.txt', 'fr':'fr.txt', 'in':'in.txt', 'ir':'ir.txt', 'pk':'pk.txt', 'za':'za.txt'}

## X = examples, y = labels
## y is its position in the languages array

## Output
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)
output, next_hidden = rnn(input[0], hidden)


##ADDED
# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

## ADDED
# Read a file and split into lines
def readLines(filename):
    lines = codecs.open(filename, "r",encoding='utf-8', errors='ignore').read().strip().split('\n')
    # lines = open(filename, encoding='utf-8', errors='ignore').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

## ADDED
def findFiles(path): return glob.glob(path)

'''
Returns the words of the language specified by reading it from the data folder
Returns the validation data if train is false and the train data otherwise.
Return: A nx1 array containing the words of the specified language
'''
def getWords(baseDir, lang, train = True):
    
  # Build the category_lines dictionary, a list of names per language
  category_lines = {}
  all_categories = []

  if train:
    datafile = 'train/*.txt'
  else:
    datafile = 'val/*.txt'


  for filename in findFiles(baseDir + datafile):
      category = os.path.splitext(os.path.basename(filename))[0]
      all_categories.append(category)
      lines = readLines(filename)
      category_lines[category] = lines

  if train:
    all_categories_train = copy.deepcopy(all_categories)
    n_categories_train = len(all_categories)
  else:
    all_categories_val = copy.deepcopy(all_categories)
    n_categories_val = len(all_categories)

  # print("category_lines[lang]: ", category_lines[lang])
  return category_lines[lang]

'''
Returns a label corresponding to the language
For example it returns an array of 0s for af
Return: A nx1 array as integers containing index of the specified language in the "languages" array
'''
def getLabels(lang, length):
    
    index = languages.index(lang)

    return [index]*length

'''
Returns all the languages and labels after reading it from the file
Returns the validation data if train is false and the train data otherwise.
You may assume that the files exist in baseDir and have the same names.
Return: X, y where X is nx1 and y is nx1
'''
def readData(baseDir, train=True):

  ## def getWords(baseDir, lang, train = True):
  ## X is examples, y is the label from  languages array
  X, y = ([],[]) 

  for label, lang in enumerate(languages):
    curr_cities = get_words(baseDir, lang, train)
    for city in curr_cities:
      X.append(city)
      y.append(label)

  return X, y


## ADDED
# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

## ADDED
# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor


'''
Convert a line/word to a pytorch tensor of numbers
Refer the tutorial in the spec
Return: A tensor corresponding to the given line
'''
def line_to_tensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

'''
Returns the category/class of the output from the neural network
Input: Output of the neural networks (class probabilities)
Return: A tuple with (language, language_index)
        language: "af", "cn", etc.
        language_index: 0, 1, etc.
'''
def category_from_output(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories_train[category_i], category_i 

## ADDED
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

'''
Get a random input output pair to be used for training 
Refer the tutorial in the spec
'''
def random_training_pair(X, y):
    category_index = randomChoice(range(len(y)))
    category_label = y[category_index]
    category = languages[category_label]
    line = X[category_index]
    category_tensor = torch.tensor([category_label], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

## ADDED 
## Test random_training_pair
for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

'''
Input: trained model, a list of words, a list of class labels as integers
Output: a list of class labels as integers
'''
def predict(model, X, y=None):
    print('\n> %s' % input_line)
    n_predictions = 3
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories_train[category_index]))
            predictions.append([value, all_categories_train[category_index]])

      return predictions

## ADDED
def elapsed_time(start):
    end = time.time()
    total = end - start
    mins = math.floor( total / 60)
    secs -= mins * 60
    return '%dm %ds' % (mins, secs)

'''
Input: trained model, a list of words, a list of class labels as integers
Output: The accuracy of the given model on the given input X and target y
'''
def calculateAccuracy(model, X, y):
    pass

'''
Train the model for one epoch/one training word.
Ensure that it runs within 3 seconds.
Input: X and y are lists of words as strings and classes as integers respectively
Returns: You may return anything
'''
def trainOneEpoch(model, criterion, optimizer, X, y):
    model.zero_grad()
    hidden = model.init_hidden()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = model(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    optimizer.step()

    return output, loss.data[0]

'''
Use this to train and save your classification model. 
Save your model with the filename "model_classify"
'''
def run():

  ## Load Data
  X_train, y_train = readData(baseDir,True)
  X_val, y_val = readData(baseDir, False)

  ## Important Variables
  n_epochs = 250000
  print_interval = 5000
  plot_interval = 1000
  accuracy = 0
  accuracy_val = 0
  current_loss = 0
  current_loss_val = 0
  all_losses = []
  all_losses_val = []

  ## Create Model
  n_hidden = 128
  rnn = RNN(n_letters, n_hidden, n_categories_train)

  ## Parameters
  criterion = nn.NLLLoss()
  ## def trainOneEpoch(model, criterion, optimizer, X, y)
  optimizer = torch.optim.RMSprop(rnn.parameters(), lr=learning_rate, momentum = 0.0075)

  start = time.time()

  for epoch in range(1, n_epochs + 1):
    # Get a random training input and target
    category, line, category_tensor, line_tensor = random_training_pair(X_train, y_train)
    output, loss = train(category_tensor, line_tensor)
    guess, guess_i = category_from_output(output)
    if guess == category:
        accuracy += 1
    current_loss += loss
    
    v_category, v_line, v_category_tensor, v_line_tensor = random_validation_pair()
    v_output, v_loss = train(v_category_tensor, v_line_tensor)
    v_guess, v_guess_i = category_from_output(v_output)
    if v_guess == v_category:
        accuracy_val += 1
    current_loss_val += v_loss
    
    
    # Print epoch number, loss, name and guess
    if epoch % print_interval == 0:
        guess, guess_i = category_from_output(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (epoch, epoch / n_epochs * 100, elapsed_time(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if epoch % plot_interval == 0:
        all_losses.append(current_loss / plot_interval)
        current_loss = 0
        
        all_losses_val.append(current_loss_val / plot_interval)
        current_loss_val = 0


    ## Save Model
    torch.save(model.state_dict(), baseDir)
    model = CharRNNClassify()
    model.load_state_dict(torch.load(baseDir))
    model.eval() #To predict


# models.py

In [0]:
#models.py
import torch.nn as nn
from torch.autograd import Variable
import torch
import torch.nn.functional as F

'''
Please add default values for all the parameters of __init__.
'''
class CharRNNClassify(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden=None):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

# Save Models

In [0]:
torch.save(model.state_dict(), PATH)

In [0]:
model = CharRNNClassify()
model.load_state_dict(torch.load(PATH))
model.eval() #To predict

# Test Set

In [0]:
lines = readLines(baseDir + '/cities_test.txt')
out_lines = []
for line in lines:
    line_tensor = Variable(line_to_tensor(line))
    output = evaluate(line_tensor)
    guess, guess_i = category_from_output(output)
    out_lines.append(guess)
    
with open('labels.txt', 'w') as f:
    for line in out_lines:
        f.write(line + '\n')
